In [ ]:
# Installs all the necessary packages for the project
%pip install pandas pandas-ta numpy matplotlib statsmodels pandas_datareader datetime yfinance scikit-learn PyPortfolioOpt
%pip install --upgrade certifi

In [2]:
# Imports all the necessary packages for the project and fixes ssl error
import ssl
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta as ta
import warnings
warnings.filterwarnings('ignore')
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
# Get SP500 data    
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

end_date = dt.datetime.now().strftime('%Y-%m-%d')
start_date = (pd.to_datetime(end_date) - pd.DateOffset(years=10)).strftime('%Y-%m-%d')

df = yf.download(tickers=symbols_list, start=start_date, end=end_date).stack()

df.index.names = ['Date', 'Symbol']

df

[*********************100%%**********************]  503 of 503 completed


Adj Close       Close        High         Low        Open  \
Date       Symbol                                                               
2014-01-27 A        38.307018   41.702431   41.752502   40.758224   41.416309   
           AAL      28.451939   30.180000   30.870001   28.670000   30.400000   
           AAPL     17.236380   19.660713   19.814285   19.491072   19.645357   
           ABBV     31.039011   46.830002   47.830002   46.419998   47.790001   
           ABT      29.952211   36.299999   36.830002   36.130001   36.680000   
...                       ...         ...         ...         ...         ...   
2024-01-26 YUM     129.089996  129.089996  130.690002  128.669998  129.919998   
           ZBH     121.690002  121.690002  123.110001  121.570000  122.839996   
           ZBRA    252.169998  252.169998  258.420013  251.619995  256.980011   
           ZION     44.020000   44.020000   44.860001   43.959999   44.500000   
           ZTS     191.520004  191.520004  192.880005  188.970001  189.210007   

                        Volume  
Date       Symbol               
2014-01-27 A         6527262.0  
           AAL      16276300.0  
           AAPL    554878800.0  
           ABBV     10749800.0  
           ABT       9128900.0  
...                        ...  
2024-01-26 YUM       1157000.0  
           ZBH        982800.0  
           ZBRA       268300.0  
           ZION      1504900.0  
           ZTS       1840200.0  

[1236287 rows x 6 columns]

In [11]:
# Calculate features and technical indicators
# Garman-Klass Volatility, RSI, Bollinger Bands, ATR, MACD, Dollar Volume,
# All but RSI are normalized by subtracting the mean and dividing by the standard deviation

df['Garman-Klass'] = ((np.log(df['High'])-np.log(df['Low']))**2)/2-(2*np.log(2)-1)*((np.log(df['Adj Close'])-np.log(df['Open']))**2)  

df['RSI'] = df.groupby(level=1)['Adj Close'].transform(lambda x: ta.rsi(close=x, length=20))

df['BB-Low'] = df.groupby(level=1)['Adj Close'].transform(lambda x: ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['BB-Mid'] = df.groupby(level=1)['Adj Close'].transform(lambda x: ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['BB-High'] = df.groupby(level=1)['Adj Close'].transform(lambda x: ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(data):
    atr = ta.atr(high=data['High'],
                        low=data['Low'],
                        close=data['Close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())
df['ATR'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['MACD'] = df.groupby(level=1, group_keys=False)['Adj Close'].apply(compute_macd)

df['Dollar Volume'] = (df['Adj Close']*df['Volume'])/1e6

df


Adj Close       Close        High         Low        Open  \
Date       Symbol                                                               
2014-01-27 A        38.307018   41.702431   41.752502   40.758224   41.416309   
           AAL      28.451939   30.180000   30.870001   28.670000   30.400000   
           AAPL     17.236380   19.660713   19.814285   19.491072   19.645357   
           ABBV     31.039011   46.830002   47.830002   46.419998   47.790001   
           ABT      29.952211   36.299999   36.830002   36.130001   36.680000   
...                       ...         ...         ...         ...         ...   
2024-01-26 YUM     129.089996  129.089996  130.690002  128.669998  129.919998   
           ZBH     121.690002  121.690002  123.110001  121.570000  122.839996   
           ZBRA    252.169998  252.169998  258.420013  251.619995  256.980011   
           ZION     44.020000   44.020000   44.860001   43.959999   44.500000   
           ZTS     191.520004  191.520004  192.880005  188.970001  189.210007   

                        Volume  Garman-Klass        RSI    BB-Low    BB-Mid  \
Date       Symbol                                                             
2014-01-27 A         6527262.0     -0.002062        NaN       NaN       NaN   
           AAL      16276300.0      0.001039        NaN       NaN       NaN   
           AAPL    554878800.0     -0.006476        NaN       NaN       NaN   
           ABBV     10749800.0     -0.071501        NaN       NaN       NaN   
           ABT       9128900.0     -0.015676        NaN       NaN       NaN   
...                        ...           ...        ...       ...       ...   
2024-01-26 YUM       1157000.0      0.000105  50.708962  4.856013  4.872770   
           ZBH        982800.0      0.000045  56.739863  4.794179  4.811249   
           ZBRA       268300.0      0.000218  51.191561  5.479828  5.545973   
           ZION      1504900.0      0.000160  57.948030  3.728311  3.786146   
           ZTS       1840200.0      0.000153  52.795286  5.230000  5.267848   

                    BB-High       ATR      MACD  Dollar Volume  
Date       Symbol                                               
2014-01-27 A            NaN       NaN       NaN     250.039945  
           AAL          NaN       NaN       NaN     463.092289  
           AAPL         NaN       NaN       NaN    9564.101642  
           ABBV         NaN       NaN       NaN     333.663160  
           ABT          NaN       NaN       NaN     273.430742  
...                     ...       ...       ...            ...  
2024-01-26 YUM     4.889527  0.169377  0.224806     149.357126  
           ZBH     4.828319 -0.523774  0.413233     119.596934  
           ZBRA    5.612118  0.287127 -0.006501      67.657211  
           ZION    3.843982  0.697197  0.630047      66.245699  
           ZTS     5.305696  0.931215 -0.322147     352.435112  

[1236287 rows x 14 columns]

In [21]:
# Aggregate to monthly data and filter top 150 most liquid stocks for each month

tech_cols = [c for c in df.columns.unique(0) if c not in ['Dollar Volume', 'Volume', 'Open',
                                                          'High', 'Low', 'Close']]

tech_data = (pd.concat([df.unstack('Symbol')['Dollar Volume'].resample('M').mean().stack('Symbol').to_frame('Dollar Volume'),
                   df.unstack()[tech_cols].resample('M').last().stack('Symbol')],
                  axis=1)).dropna()

# Calculate 5-year rolling average of Dollar Volume
tech_data['Dollar Volume'] = (tech_data.loc[:, 'Dollar Volume'].unstack('Symbol').rolling(5*12, min_periods=12).mean().stack())

tech_data['DV Rank'] = (tech_data.groupby('Date')['Dollar Volume'].rank(ascending=False))

# Filter top 150 most liquid stocks for each month and put in new dataframe
#tech_data = tech_data[tech_data['DV Rank']<150].drop(['Dollar Volume', 'DV Rank'], axis=1)
top_150 = tech_data[tech_data['DV Rank'] < 150]

top_150


Dollar Volume   Adj Close  Garman-Klass        RSI  \
Date       Symbol                                                       
2015-02-28 AAL        489.261940   45.477360     -0.001642  45.417051   
           AAPL      5126.441674   28.860920     -0.005285  62.555779   
           ABBV       371.581301   41.377697     -0.055845  48.492200   
           ABT        195.715541   39.949791     -0.011462  61.637691   
           ACN        189.507278   77.266312     -0.009006  55.963665   
...                          ...         ...           ...        ...   
2024-01-31 VRTX       392.203975  430.170013      0.000043  64.519320   
           VZ         790.422950   42.400002      0.000040  70.186636   
           WFC        992.283710   50.320000      0.000071  63.682813   
           WMT        976.278400  164.270004      0.000034  60.845739   
           XOM       1415.004120  103.000000      0.000132  55.632470   

                     BB-Low    BB-Mid   BB-High       ATR      MACD  DV Rank  
Date       Symbol                                                             
2015-02-28 AAL     3.807215  3.858552  3.909890  1.977849 -0.194001     35.0  
           AAPL    3.289049  3.368803  3.448558 -0.777060  0.276303      1.0  
           ABBV    3.660831  3.721655  3.782480 -0.699865 -0.423286     51.0  
           ABT     3.641809  3.685499  3.729188 -0.906787  0.367600    116.0  
           ACN     4.306899  4.345959  4.385019 -0.964695 -0.040922    120.0  
...                     ...       ...       ...       ...       ...      ...  
2024-01-31 VRTX    6.005536  6.054672  6.103808  0.960355  2.704602    116.0  
           VZ      3.621979  3.697084  3.772190  0.192408  2.633689     45.0  
           WFC     3.862753  3.907793  3.952834 -0.340065  0.644597     33.0  
           WMT     5.058206  5.084690  5.111174  0.135625  1.046672     34.0  
           XOM     4.570105  4.613145  4.656185  0.246282 -0.437450     17.0  

[16092 rows x 10 columns]

In [24]:
# Calculate monthly returns for different time horizons on the top 150 most liquid stocks
def calculate_returns(df):
    outlier_cutoff = 0.005
    lags = [1, 2, 3, 6, 9, 12]
    for lag in lags:
        df[f'{lag}m Return'] = (df['Adj Close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df

returns = top_150.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()
returns

Dollar Volume   Adj Close  Garman-Klass        RSI  \
Date       Symbol                                                       
2016-02-29 AAL        479.985129   39.288376     -0.000319  54.607062   
           AAPL      5210.567829   22.125891     -0.003013  46.730995   
           ABBV       404.930430   38.690907     -0.051766  48.010303   
           ABT        216.240057   33.398453     -0.009964  46.271609   
           ACN        209.341639   87.900642     -0.006994  49.785694   
...                          ...         ...           ...        ...   
2024-01-31 VRTX       392.203975  430.170013      0.000043  64.519320   
           VZ         790.422950   42.400002      0.000040  70.186636   
           WFC        992.283710   50.320000      0.000071  63.682813   
           WMT        976.278400  164.270004      0.000034  60.845739   
           XOM       1415.004120  103.000000      0.000132  55.632470   

                     BB-Low    BB-Mid   BB-High       ATR      MACD  DV Rank  \
Date       Symbol                                                              
2016-02-29 AAL     3.547833  3.638645  3.729458  0.676352  0.535499     39.0   
           AAPL    3.106253  3.130723  3.155194 -0.829064 -0.352313      1.0   
           ABBV    3.629341  3.677365  3.725388 -0.332450 -0.284925     49.0   
           ABT     3.479070  3.522661  3.566251 -0.585796 -0.392764    113.0   
           ACN     4.405534  4.474354  4.543174 -0.604643 -0.296978    121.0   
...                     ...       ...       ...       ...       ...      ...   
2024-01-31 VRTX    6.005536  6.054672  6.103808  0.960355  2.704602    116.0   
           VZ      3.621979  3.697084  3.772190  0.192408  2.633689     45.0   
           WFC     3.862753  3.907793  3.952834 -0.340065  0.644597     33.0   
           WMT     5.058206  5.084690  5.111174  0.135625  1.046672     34.0   
           XOM     4.570105  4.613145  4.656185  0.246282 -0.437450     17.0   

                   1m Return  2m Return  3m Return  6m Return  9m Return  \
Date       Symbol                                                          
2016-02-29 AAL      0.054421  -0.014726  -0.001198   0.009277  -0.002847   
           AAPL    -0.001288  -0.038976  -0.063339  -0.023725  -0.031060   
           ABBV    -0.005283  -0.034844  -0.017302  -0.018800  -0.018854   
           ABT      0.023514  -0.068304  -0.046141  -0.023736  -0.023047   
           ACN     -0.050028  -0.020497  -0.022124   0.012121   0.005983   
...                      ...        ...        ...        ...        ...   
2024-01-31 VRTX     0.057214   0.101088   0.059091   0.033823   0.026237   
           VZ       0.124629   0.060583   0.064688   0.043639   0.016077   
           WFC      0.022349   0.062311   0.084792   0.017307   0.029339   
           WMT      0.041992   0.029091   0.002990   0.005761   0.010665   
           XOM      0.030206   0.001265  -0.006045  -0.003838  -0.012469   

                   12m Return  
Date       Symbol              
2016-02-29 AAL      -0.012116  
           AAPL     -0.021902  
           ABBV     -0.005579  
           ABT      -0.014815  
           ACN       0.010804  
...                       ...  
2024-01-31 VRTX      0.024138  
           VZ        0.007677  
           WFC       0.008523  
           WMT       0.012388  
           XOM      -0.007089  

[13801 rows x 16 columns]

In [29]:
# Use Fama—French data to estimate the exposure of assets to common risk factors using linear regression.

factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start=start_date, end=end_date)[0].drop('RF', axis=1)
factor_data.index = factor_data.index.to_timestamp()
factor_data.index.name = 'Date'
factor_data = factor_data.resample('M').last().div(100)
factor_data = factor_data.join(returns['1m Return']).sort_index()


# Filter out stocks with less than 10 months of data.
observations = factor_data.groupby(level=1).size()
valid_stocks = observations[observations >= 10]
factor_data = factor_data[factor_data.index.get_level_values('Symbol').isin(valid_stocks.index)]

factor_data


Mkt-RF     SMB     HML     RMW     CMA  1m Return
Date       Symbol                                                   
2016-02-29 AAL    -0.0007  0.0088 -0.0057  0.0325  0.0202   0.054421
           AAPL   -0.0007  0.0088 -0.0057  0.0325  0.0202  -0.001288
           ABBV   -0.0007  0.0088 -0.0057  0.0325  0.0202  -0.005283
           ABT    -0.0007  0.0088 -0.0057  0.0325  0.0202   0.023514
           ACN    -0.0007  0.0088 -0.0057  0.0325  0.0202  -0.050028
...                   ...     ...     ...     ...     ...        ...
2023-11-30 VRTX    0.0884 -0.0010  0.0165 -0.0389 -0.0099  -0.020160
           VZ      0.0884 -0.0010  0.0165 -0.0389 -0.0099   0.091090
           WFC     0.0884 -0.0010  0.0165 -0.0389 -0.0099   0.131192
           WMT     0.0884 -0.0010  0.0165 -0.0389 -0.0099  -0.047243
           XOM     0.0884 -0.0010  0.0165 -0.0389 -0.0099  -0.020506

[13474 rows x 6 columns]

In [31]:
# Calculate Rolling Factor Betas

betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['1m Return'], 
                                     exog=sm.add_constant(x.drop('1m Return', axis=1)),
                                     window=min(24, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params
         .drop('const', axis=1)))

#betas 

# Join betas to returns dataframe
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
data = (returns.join(betas.groupby('Symbol').shift()))
data.loc[:, factors] = data.groupby('Symbol', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))
#data = data.drop('adj close', axis=1)
data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 13545 entries, (Timestamp('2016-02-29 00:00:00'), 'AAL') to (Timestamp('2024-01-31 00:00:00'), 'XOM')
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Dollar Volume  13545 non-null  float64
 1   Adj Close      13545 non-null  float64
 2   Garman-Klass   13545 non-null  float64
 3   RSI            13545 non-null  float64
 4   BB-Low         13545 non-null  float64
 5   BB-Mid         13545 non-null  float64
 6   BB-High        13545 non-null  float64
 7   ATR            13545 non-null  float64
 8   MACD           13545 non-null  float64
 9   DV Rank        13545 non-null  float64
 10  1m Return      13545 non-null  float64
 11  2m Return      13545 non-null  float64
 12  3m Return      13545 non-null  float64
 13  6m Return      13545 non-null  float64
 14  9m Return      13545 non-null  float64
 15  12m Return     13545 non-null  float64
 16  Mkt-RF         135